In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install accelerate>=0.12.0
!pip install datasets
!pip install evaluate
!pip install sacrebleu

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/T5

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate
import numpy as np

checkpoint = "T5-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
new_words = ['{', '}', "<"]
tokenizer.add_tokens(new_words)

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint,return_dict=True)
model.resize_token_embeddings(len(tokenizer))

prefix = "translate English to Sparql: "
metric = evaluate.load("sacrebleu")


def preprocess_function(examples):
    inputs = []
    targets = []
    for i, question in enumerate(examples["question"]):
        question1 = question["string"]
        # question2 = examples["paraphrased_question"][i]["string"]
        answer = examples["query"][i]["sparql"]
        entities = examples["entities"][i]
        relations = examples["relations"][i]
        inputs.append(prefix + question1 + "\nentities: " + str(entities) + "\nrelations: " + str(relations))
        # inputs.append(prefix + question2 + "\nentities: " + str(entities) + "\nrelations: " + str(relations))
        targets.append(answer)
        # targets.append(answer)
    model_inputs = tokenizer(inputs, text_target=targets, max_length=512, truncation=True)
    return model_inputs


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result


data = load_dataset("awalesushil/DBLP-QuAD")
print(data)
tokenized_books = data.map(preprocess_function, batched=True)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
    output_dir="en2sparql_T5_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=20,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_books["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save_model()